<a href="https://colab.research.google.com/github/bernaldiaz/curso-numerico-3/blob/main/T12Aproximaci%C3%B3nSolucionesSistemasNoLineales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 12 - Aproximación a las Soluciones de Sistemas de Ecuaciones no Lineales

## Librerías y Funciones Necesarias

In [ ]:
import numpy as np
import math
import pandas as pd
pd.set_option('max_columns', None)

## Método del Punto Fijo

La idea es que si $p$ es un **punto fijo** de la función de varias variables $G(x)$, entonces $p$ necesariamente debe ser un cero de nuestra función de varias variables $F(x)$, ya que $F(x) = 0$ es equivalente a $G(x) = x$

In [ ]:
def PuntoFijo(n, G, x0, TOL, nMax, verbose = False):
    """
    Devuelve una solución aproximada al sistema de ecuaciones no lineales 
    g_i(x) = x_i para i = 1,..., n
    
    Args:
        n: (int) Número de ecuaciones del sistema de ecuaciones no lineal y
            número de variables
        G: (ndarray) Array de n funciones g_i(x), componentes de G(x)
        x0: (ndarray) Array de valores iniciales
        TOL: (float) Tolerancia
        nMax: (int) Número máximo de iteraciones
        verbose: (bool) Para mostrar o no los resultados relevantes
        
    Returns:
        x: (ndarray) Array de soluciones aproximadas
        
    """

    k = 1 # Inicializamos el contador de las iteraciones

    while (k <= nMax):
        x = np.zeros(n) # Inicializamos la lista que contiene los valores x_i
        for i in range(n): # Calculamos el siguiente valor de la sucesión
            x[i] = G[i](x0)

        if verbose:
            print("\n\nIteración {}".format(k))
            print("x =", x)
        
        # Calculamos el error entre los dos términos consecutivos de la sucesión
        error = np.linalg.norm(x - x0) 

        if verbose:
            print("error =", round(error, 7))

        if error < TOL:
            if verbose:
                print("\n\nLa solución aproximada vale {}".format(x))
            return x
        
        x0 = x.copy() # Actualizamos los términos de la sucesión
        k += 1

    if verbose:
        print("El método no converge")
    return


In [ ]:
def g1(x):
    return 1 / 5 * math.sin(x[0] * x[1]) + x[0] / 5 + 1

def g2(x):
    return 1 / 5 * math.exp(-x[0] * x[1])

G = np.array([g1, g2])

x0 = np.array([1.5, 0.5])
TOL = 0.00001
nMax = 15

In [ ]:
x = PuntoFijo(2, G, x0, TOL, nMax, verbose = True)



Iteración 1
x = [1.43632775 0.09447331]
error = 0.4104949


Iteración 2
x = [1.31432127 0.17462184]
error = 0.1459773


Iteración 3
x = [1.30836418 0.15898473]
error = 0.0167334


Iteración 4
x = [1.30297546 0.16243979]
error = 0.0064012


Iteración 5
x = [1.30261076 0.16184876]
error = 0.0006945


Iteración 6
x = [1.30237567 0.16198301]
error = 0.0002707


Iteración 7
x = [1.30235541 0.16196085]
error = 3e-05


Iteración 8
x = [1.30234507 0.16196606]
error = 1.16e-05


Iteración 9
x = [1.302344   0.16196523]
error = 1.4e-06


La solución aproximada vale [1.302344   0.16196523]


## Método de Newton-Raphson

El método de Newton-Raphson hace uso de la función de iteración 

$$G(x) = x - J(x)^{-1}F(x)$$

quedando la sucesión $x^{(k)}$ definida por

$$x^{(k)} = G(x^{(k - 1)}) = x^{(k - 1)} - J(x^{(k - 1)})^{-1}F(x^{(k - 1)})$$

In [ ]:
def NewtonRaphson(n, F, J, x0, TOL, nMax, verbose = False):
    """
    Devuelve una solución aproximada al sistema de ecuaciones no lineales 
    f_i(x) = 0 para i = 1,..., n
    
    Args:
        n: (int) Número de ecuaciones del sistema de ecuaciones no lineal y
            número de variables
        F: (ndarray) Array de n funciones f_i(x), componentes de F(x)
        J: (ndarray)  Matriz Jacobiana de F
        x0: (ndarray) Array de valores iniciales
        TOL: (float) Tolerancia
        nMax: (int) Número máximo de iteraciones
        verbose: (bool) Para mostrar o no los resultados relevantes
        
    Returns:
        x: (ndarray) Array de soluciones aproximadas
        
    """

    k = 1 # Inicializamos el contador de las iteraciones

    F0 = np.zeros_like(F, dtype = float)
    J0 = np.zeros_like(J, dtype = float)
    while (k <= nMax):
        for i in range(n): 
            # Calculamos todas las componentes de la función F en x0
            F0[i] = F[i](x0)

            # Calculamos la matriz jacobiana en x0
            for j in range(n):
                J0[i, j] = J[i, j](x0)

        if verbose:
            print("\n\nIteración {}".format(k))
            print("F0 =", F0)
            print("J0 =\n", J0)

        # Resolvemos el sistema lineal J(x0)y = -F(x0)
        y = np.linalg.solve(J0, -F0)
        x0 = x0 + y.copy()

        if verbose:
            print("x =", x0)

        # El error entre los dos términos consecutivos vale la solución del sistema
        error = np.linalg.norm(y)

        if verbose:
            print("error =", round(error, 7))

        if error < TOL:
            if verbose:
                print("\n\nLa solución aproximada vale {}".format(x0))
            return x0

        k += 1 

    if verbose:
        print("El método no converge")
    return


In [ ]:
# Definición de F
def f1(x):
    return math.pow(x[0], 2) + math.pow(x[1], 2) - math.pow(x[2], 2) - 2 

def f2(x):
    return math.sin(x[0]) - math.cos(x[1]) + math.exp(x[2])

def f3(x):
    return x[0] * x[1] * x[2] - 1

F = np.array([f1, f2, f3])

# Definición de J
def j11(x):
    return 2 * x[0]

def j12(x):
    return 2 * x[1]

def j13(x):
    return -2 * x[2]

def j21(x):
    return math.cos(x[0])

def j22(x):
    return math.sin(x[1])

def j23(x):
    return math.exp(x[2])

def j31(x):
    return x[1] * x[2]

def j32(x):
    return x[0] * x[2]

def j33(x):
    return x[0] * x[1] 

J = np.array([[j11, j12, j13],
             [j21, j22, j23],
             [j31, j32, j33]])

# Resto de parámetros
x0 = np.array([1, 1, 1])
TOL = 0.000001
nMax = 25

In [ ]:
x = NewtonRaphson(3, F, J, x0, TOL, nMax, verbose = True)



Iteración 1
F0 = [-1.          3.01945051  0.        ]
J0 =
 [[ 2.          2.         -2.        ]
 [ 0.54030231  0.84147098  2.71828183]
 [ 1.          1.          1.        ]]
x = [ 9.46783871 -7.21783871  0.75      ]
error = 11.8025279


Iteración 2
F0 = [139.17466534   1.47985531 -52.2529995 ]
J0 =
 [[ 18.93567741 -14.43567741  -1.5       ]
 [ -0.99907303  -0.80439321   2.11700002]
 [ -5.41337903   7.10087903 -68.33733267]]
x = [ 6.48380404 -1.49792721  0.81610019]
error = 6.4518385


Iteración 3
F0 = [41.61748118  2.38813361 -8.92618255]
J0 =
 [[12.96760807 -2.99585442 -1.63220038]
 [ 0.97994347 -0.99734622  2.26166257]
 [-1.22245868  5.29143372 -9.7122665 ]]
x = [4.52885712 2.97169332 2.57823883]
error = 5.1869507


Iteración 4
F0 = [20.69419255 13.17631274 33.69890358]
J0 =
 [[ 9.05771424  5.94338664 -5.15647766]
 [-0.18250325  0.16908313 13.1739162 ]
 [ 7.6617351  11.67647528 13.45837445]]
x = [1.4019603  3.34569037 1.52993869]
error = 3.3190798


Iteración 5
F0 = [8.8184243

## Algoritmo del Gradiente Descendente

In [ ]:
def g(x, F):
    """
    Función g de la que queremos hallar un mínimo

    Args:
        x: (ndarray) Array de n variables x1, x2, ..., xn
        F: (ndarray) Array de n funciones f_i(x), componentes de F(x)
    """
    
    n = F.shape[0]
    y = 0 # Inicializamos el valor de g(x)
    for i in range(n):
        y += math.pow(F[i](x), 2)
    return y

In [ ]:
def grad_g(x, F, J):
    """
    Calcula el gradiente de la función g en un punto x

    Args:
        x: (ndarray) Array de n variables x1, x2, ..., xn
        F: (ndarray) Array de n funciones f_i(x), componentes de F(x)
        J: (ndarray)  Matriz Jacobiana de F
    """

    n = F.shape[0]
    grad = np.zeros(n) # Inicializamos el vector gradiente a 0
    for k in range(n):
        for i in range(n):
            grad[k] += J[i, k](x) * F[i](x)

    return 2 * grad

In [ ]:
def GradienteDescendente(n, F, J, x0, TOL, nMax, maxIter, verbose = False):
    """
    Devuelve una solución aproximada al sistema de ecuaciones no lineales 
    f_i(x) = 0 para i = 1,..., n
    
    Args:
        n: (int) Número de ecuaciones del sistema de ecuaciones no lineal y
            número de variables
        F: (ndarray) Array de n funciones f_i(x), componentes de F(x)
        J: (ndarray)  Matriz Jacobiana de F
        x0: (ndarray) Array de valores iniciales
        TOL: (float) Tolerancia
        nMax: (int) Número máximo de iteraciones
        maxIter: (int) Número máximo de iteraciones a mostrar por consola
        verbose: (bool) Para mostrar o no los resultados relevantes
        
    Returns:
        x: (ndarray) Array de soluciones aproximadas
        
    """

    k = 1 # Inicializamos el contador de las iteraciones

    while (k <= nMax):
        g1 = g(x0, F)
        gradX = grad_g(x0, F, J)
        normX = np.linalg.norm(gradX)

        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("\n\nIteración {}".format(k))
            print("g1 =", g1)
            print("gradiente de g =\n", gradX)

        if normX == 0:
            if verbose:
                print("Hemos encontrado un valor de gradiente cero.\nComprobar si x es una aproximación de un cero de la función F(x)")
                print("x =", x0)
            return x0

        gradX /= normX # Normalizamos el gradiente
        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("gradiente de g normalizado =\n", gradX)

        # Inicializamos las u_i
        u1 = 0 
        u3 = 1
        g3 = g(x0 - u3 * gradX, F)
        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("u1 =", u1)
            print("u3 =", u3)
            print("g3 =", g3)
        
        # Refinamos el valor u3
        while g3 >= g1:
            u3 /= 2
            g3 = g(x0 - u3 * gradX, F)

            if u3 < TOL / 2:
                if verbose:
                    print("No hay mejora significativa.\nComprobar si x es una aproximación de un cero de la función F(x)")
            
        u2 = u3 / 2
        g2 = g(x0 - u2 * gradX, F)
        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("u2 =", u2)
            print("g2 =", g2)

        h1 = (g2 - g1) / u2 # Hallamos el mínimo del polinomio de interpolación P2(u)
        h2 = (g3 - g2) / (u3 - u2)
        h3 = (h2 - h1) / u3

        um = 1/2 * (u2 - h1 / h3)
        gm = g(x0 - um * gradX, F)

        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("h1 =", h1)
            print("h2 =", h2)
            print("h3 =", h3)
            print("um =", um)
            print("gm =", gm)

        if (g3 < gm): # Calculamos el mínimo entre g3 y gm
            um = u3
            gm = g3

        x = x0 - um * gradX # Hallamos el siguiente término de la sucesión
        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("x =", x)
        
        error = np.linalg.norm(x - x0)
        if verbose and (k <= maxIter or k >= (nMax - 5)):
            print("error =", round(error, 7))

        if error < TOL:
            if verbose:
                print("\n\nNúmero total de iteraciones:", k)
                print("Solución aproximada x =", x)
            return x

        x0 = x.copy()
        k += 1

            
    if verbose:
        print("El método no converge")
    return

In [ ]:
# Definición de F
def f1(x):
    return math.pow(x[0], 2) + math.pow(x[1], 2) - math.pow(x[2], 2) - 2 

def f2(x):
    return math.sin(x[0]) - math.cos(x[1]) + math.exp(x[2])

def f3(x):
    return x[0] * x[1] * x[2] - 1

F = np.array([f1, f2, f3])

# Definición de J
def j11(x):
    return 2 * x[0]

def j12(x):
    return 2 * x[1]

def j13(x):
    return -2 * x[2]

def j21(x):
    return math.cos(x[0])

def j22(x):
    return math.sin(x[1])

def j23(x):
    return math.exp(x[2])

def j31(x):
    return x[1] * x[2]

def j32(x):
    return x[0] * x[2]

def j33(x):
    return x[0] * x[1] 

J = np.array([[j11, j12, j13],
             [j21, j22, j23],
             [j31, j32, j33]])

# Resto de parámetros
x0 = np.array([1, 1, 1])
TOL = 0.000001
nMax = 300

In [ ]:
x = GradienteDescendente(3, F, J, x0, TOL, nMax, 20, verbose = True)



Iteración 1
g1 = 10.117081366630883
gradiente de g =
 [-0.73716786  1.08155998 20.41543489]
gradiente de g normalizado =
 [-0.03603438  0.05286902  0.9979511 ]
u1 = 0
u3 = 1
g3 = 2.6312325196258084
u2 = 0.5
g2 = 4.084850695718863
h1 = -12.064461341824039
h2 = -2.9072363521861098
h3 = 9.157224989637928
um = 0.908740030712135
gm = 2.7205323606986607
x = [1.03603438 0.94713098 0.0020489 ]
error = 1.0


Iteración 2
g1 = 2.6312325196258084
gradiente de g =
 [1.17660863 1.95920745 0.60376485]
gradiente de g normalizado =
 [0.4977667  0.82884674 0.25542396]
u1 = 0
u3 = 1
g3 = 4.2193045131901386
u2 = 0.125
g2 = 2.449157606806254
h1 = -1.4565993025564339
h2 = 0.08890720516137307
h3 = 6.182026030871228
um = 0.1803092178262113
gm = 2.433147667415008
x = [ 0.94628245  0.79768228 -0.04400639]
error = 0.1803092


Iteración 3
g1 = 2.433147667415008
gradiente de g =
 [-0.45612533  0.11724136  0.40496082]
gradiente de g normalizado =
 [-0.73436032  0.18875821  0.65198563]
u1 = 0
u3 = 1
g3 = 3.8892712